In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

%matplotlib inline

In [3]:
# Define some helper functions
def categorical_lookup(series):
    return dict(zip(series, np.arange(1, len(series) + 1)))

In [4]:
categorical_lookup(('a', 'b', 'c', 'd'))

{'a': 1, 'b': 2, 'c': 3, 'd': 4}